In [1]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
# 데이터프레임 읽기
df = pd.read_csv('./data/weather_kor4.csv')

In [3]:
df.isna().sum()

Unnamed: 0    0
날짜            0
기온            0
강수량           0
풍속_num        0
풍향_num        0
습도            0
현지기압          0
전운량           0
aplYmd        0
날씨코드          0
wetrTxt       0
wetrSplit     0
현상번호_split    0
weather_1     0
weather_2     0
풍속            0
풍향            0
일조량           0
시간당 강수량       0
timestamp     0
W1            0
W2            0
dtype: int64

In [4]:
df.drop(columns={'W1','W2'},inplace=True)

In [5]:
df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
df["시간당 강수량"] = df["시간당 강수량"].fillna("없음")
# 데이터프레임 nan값 지우기
df = df.dropna()

In [6]:
# 인덱스제거
df.reset_index(drop=True,inplace=True)
# weather_1에 라벨인코딩
encoder = LabelEncoder()
encoder.fit(df["weather_1"])
encoded_data = encoder.transform(df["weather_1"])
# # weather_2에 라벨인코딩
encoder1 = LabelEncoder()
encoder1.fit(df["weather_2"])
encoded_data1 = encoder1.transform(df["weather_2"])
# data_Frame으로 변환
encoded_data_df = pd.DataFrame(encoded_data,columns=['W1'])
# data_Frame으로 변환
encoded_data_df1 = pd.DataFrame(encoded_data1,columns=['W2'])

In [7]:
from datetime import datetime
df['timestamp'] = pd.to_datetime(df['날짜'])
# data_frame 통합
df = pd.concat([df,encoded_data_df,encoded_data_df1],axis=1)

In [8]:
input_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']
output_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']

In [9]:
df

,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,날씨코드,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1
1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,20191229,9.0,...,"['92', '01']",구름많음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 01:00:00,0,1
2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 02:00:00,1,1
3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 03:00:00,1,1
4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 04:00:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37690,2024-05-23 19:00,22.5,0.0,3.5,270.0,55,999.5,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,W,1.0,없음,2024-05-23 19:00:00,1,1
37691,2024-05-23 20:00,20.5,0.0,3.0,290.0,68,1000.3,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,0.0,없음,2024-05-23 20:00:00,1,1
37692,2024-05-23 21:00,18.6,0.0,2.8,270.0,73,1001.0,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,W,-1.0,없음,2024-05-23 21:00:00,1,1
37693,2024-05-23 22:00,17.7,0.0,2.9,290.0,79,1001.6,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,-1.0,없음,2024-05-23 22:00:00,1,1


In [10]:
df.columns

Index(['날짜', '기온', '강수량', '풍속_num', '풍향_num', '습도', '현지기압', '전운량', 'aplYmd',
       '날씨코드', 'wetrTxt', 'wetrSplit', '현상번호_split', 'weather_1', 'weather_2',
       '풍속', '풍향', '일조량', '시간당 강수량', 'timestamp', 'W1', 'W2'],
      dtype='object')

In [11]:
# 데이터 전처리 변수마다 범위가 다르므로 다르게 적용
from sklearn.preprocessing import MinMaxScaler
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df[feature] = scaler.fit_transform(df[[feature]])
    # print(scaled_data)
    scalers[feature] = scaler
# 시퀀스 데이터 생성
sequence_length = 720  # 예측에 사용할 시퀀스 길이
X1 = []
Y1 = []
for i in tqdm(range(len(df) - sequence_length)):
    # 각 변수에 해당하는 데이터를 스케일링하여 저장
    scaled_features = []
    for feature in input_features:
        scaled_feature = scalers[feature].transform(df[[feature]].iloc[i:i+sequence_length])
        scaled_features.append(scaled_feature)
    # 각 변수를 열로 가지는 2차원 배열로 변환하여 X에 추가
    X1.append(np.concatenate(scaled_features, axis=1))
    # 출력 변수는 스케일링할 필요가 없으므로 그대로 추가
    Y1.append(df[output_features].iloc[i+sequence_length])
X2 = np.array(X1)
Y2 = np.array(Y1)


100%|█████████████████████████████████████████████████████████████████████████████| 36975/36975 [07:37<00:00, 80.74it/s]


In [12]:
train_length = int(len(X2)*0.8)
x_train = X2[0:train_length]
y_train = Y2[0:train_length]

In [13]:
y_train.shape

(29580, 9)

In [14]:
x_train.shape

(29580, 720, 9)

In [15]:
x_test = X2[train_length:]
y_test = Y2[train_length:]

In [16]:
x_test.shape

(7395, 720, 9)

In [17]:
valid_length = int(len(x_train)*0.8)
valid_length

23664

In [18]:
new_x_train = x_train[0:valid_length]
new_y_train = y_train[0:valid_length]

In [19]:
x_val = x_train[valid_length:]
y_val = y_train[valid_length:]

In [20]:
x_train.shape

(29580, 720, 9)

In [21]:
x_val.shape

(5916, 720, 9)

In [22]:
x_test.shape

(7395, 720, 9)

In [23]:
import tensorflow as tf

2024-05-27 17:55:31.161044: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 17:55:31.223866: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 17:55:31.224953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 17:55:32.398985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
# LSTM 모델 구성
# 50은 뉴런의 수 조절해야하는 파라미터 과적합 조심!
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output_features))
])

In [25]:
# 모델 컴파일
model.compile(loss='mse', optimizer='adam')

In [26]:
# 모델 조기종료
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=20,        
    restore_best_weights=True  
)

In [27]:
x_train.shape

(29580, 720, 9)

In [28]:
# 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping_callback]  
)

Epoch 1/100


2024-05-27 17:55:36.107439: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 766713600 exceeds 10% of free system memory.


925/925 [==============================] - 330s 354ms/step - loss: 0.0574 - val_loss: 0.0406
Epoch 2/100
925/925 [==============================] - 324s 350ms/step - loss: 0.0378 - val_loss: 0.0325
Epoch 3/100
925/925 [==============================] - 321s 347ms/step - loss: 0.0345 - val_loss: 0.0330
Epoch 4/100
925/925 [==============================] - 326s 352ms/step - loss: 0.0330 - val_loss: 0.0312
Epoch 5/100
925/925 [==============================] - 328s 355ms/step - loss: 0.0312 - val_loss: 0.0290
Epoch 6/100
925/925 [==============================] - 325s 351ms/step - loss: 0.0296 - val_loss: 0.0274
Epoch 7/100
925/925 [==============================] - 323s 349ms/step - loss: 0.0280 - val_loss: 0.0270
Epoch 8/100
925/925 [==============================] - 321s 347ms/step - loss: 0.0273 - val_loss: 0.0260
Epoch 9/100
925/925 [==============================] - 327s 353ms/step - loss: 0.0268 - val_loss: 0.0251
Epoch 10/100
925/925 [==============================] - 322s 348ms/

In [29]:
# 모델 평가
loss = model.evaluate(x_test, y_test)
print("Test Loss:", loss)

232/232 [==============================] - 26s 114ms/step - loss: 0.0284
Test Loss: 0.0283743217587471


In [30]:
last_date = df['timestamp'].iloc[-1]  
next_date = last_date + timedelta(days=1) 
print("다음 날짜:", next_date)

다음 날짜: 2024-05-24 23:00:00


In [31]:
last_sequence = df[input_features].iloc[-sequence_length:, :]
next_day_prediction = model.predict(np.array([last_sequence]))
np.set_printoptions(suppress=True)
for feature, scaler in scalers.items():
    if feature in output_features:
        next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
print("다음 날 기상 예측:", next_day_prediction)


1/1 [==============================] - 1s 532ms/step
다음 날 기상 예측: [[   7.4279084     0.11416399    3.1583736   247.26433      38.44667
  1014.9859        4.8776703     0.85280895    1.0058229 ]]


In [32]:
# model.save("lstm_model.h5")

In [33]:
# 다음 날의 기상 예측
last_sequence = df[input_features].iloc[-sequence_length:, :]
next_day_predictions = []
for i in tqdm(range(72)):
    next_day_prediction = model.predict(np.array([last_sequence]))
    for feature, scaler in scalers.items():
        if feature in output_features:
            next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
    next_day_predictions.append(next_day_prediction.flatten())
    last_sequence = np.vstack([last_sequence[1:], next_day_prediction])
print("72시간 동안의 기상 예측:")
for i, prediction in enumerate(next_day_predictions, start=1):
    print(f"시간 {i}: {prediction}")


  0%|                                                                                            | 0/72 [00:00<?, ?it/s]

1/1 [==============================] - 0s 54ms/step


  3%|██▎                                                                                 | 2/72 [00:00<00:05, 12.96it/s]

1/1 [==============================] - 0s 52ms/step


  6%|████▋                                                                               | 4/72 [00:00<00:05, 13.05it/s]

...

1/1 [==============================] - 0s 51ms/step


 97%|████████████████████████████████████████████████████████████████████████████████▋  | 70/72 [00:05<00:00, 13.38it/s]

1/1 [==============================] - 0s 51ms/step


100%|███████████████████████████████████████████████████████████████████████████████████| 72/72 [00:05<00:00, 13.23it/s]

72시간 동안의 기상 예측:
시간 1: [   7.4279084     0.11416399    3.1583736   247.26433      38.44667
 1014.9859        4.8776703     0.85280895    1.0058229 ]
시간 2: [   8.817515      0.32357308    4.4628696   313.5425       34.83624
 1011.8026        4.5650287     0.9142819     1.006448  ]
시간 3: [   7.6412263     0.2748068     4.3227353   301.55862      32.975742
 1013.6764        4.467952      0.84067786    1.0069792 ]
시간 4: [   7.479203      0.27322516    3.9854906   306.9318       27.383577
 1014.28094       3.4697082     0.9393071     1.0126568 ]
시간 5: [   6.844986      0.24540123    3.9039004   322.54037      22.49255
 1013.7497        3.288783      0.95497704    1.0124645 ]
시간 6: [   5.9380717     0.19916563    3.9598536   327.5447       22.293613
 1013.63617       3.6603146     0.8979106     1.0069888 ]
시간 7: [   4.947206      0.15232678    3.9714403   328.77847      23.703331
 1013.7795        3.7911057     0.8520039     1.0024996 ]
시간 8: [   3.7540402     0.0963564     3.9446745   329.74

In [ ]:
# 다음 날의 기상 예측
last_sequence = df[input_features].iloc[-sequence_length:, :]
next_day_predictions = []
for i in tqdm(range(100)):
    next_day_prediction = model.predict(np.array([last_sequence]))
    for feature, scaler in scalers.items():
        if feature in output_features:
            next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
    next_day_predictions.append(next_day_prediction.flatten())
    last_sequence = np.vstack([last_sequence[1:], next_day_prediction])
print("72시간 동안의 기상 예측:")
for i, prediction in enumerate(next_day_predictions, start=1):
    print(f"시간 {i}: {prediction}")


In [34]:
from sklearn.metrics import accuracy_score
# 모델의 예측 생성
y_pred = model.predict(x_test)
# 모델의 예측값을 클래스로 변환 (예: 0 또는 1)
y_pred_class = np.argmax(y_pred, axis=1)

# 정확도 계산
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_class)
print("Accuracy:", accuracy)

232/232 [==============================] - 28s 120ms/step
Accuracy: 0.4549019607843137


In [35]:
next_day_predictions

[array([   7.4279084 ,    0.11416399,    3.1583736 ,  247.26433   ,
          38.44667   , 1014.9859    ,    4.8776703 ,    0.85280895,
           1.0058229 ], dtype=float32),
 array([   8.817515  ,    0.32357308,    4.4628696 ,  313.5425    ,
          34.83624   , 1011.8026    ,    4.5650287 ,    0.9142819 ,
           1.006448  ], dtype=float32),
 array([   7.6412263 ,    0.2748068 ,    4.3227353 ,  301.55862   ,
          32.975742  , 1013.6764    ,    4.467952  ,    0.84067786,
           1.0069792 ], dtype=float32),
 array([   7.479203  ,    0.27322516,    3.9854906 ,  306.9318    ,
          27.383577  , 1014.28094   ,    3.4697082 ,    0.9393071 ,
           1.0126568 ], dtype=float32),
 array([   6.844986  ,    0.24540123,    3.9039004 ,  322.54037   ,
          22.49255   , 1013.7497    ,    3.288783  ,    0.95497704,
           1.0124645 ], dtype=float32),
 array([   5.9380717 ,    0.19916563,    3.9598536 ,  327.5447    ,
          22.293613  , 1013.63617   ,    3.6603146 ,

In [36]:
next_day_prediction

array([[  -1.1087496 ,   -0.09424423,    4.1284986 ,  329.96304   ,
          42.107567  , 1013.06      ,    4.8663163 ,    0.52094144,
           0.95570403]], dtype=float32)

In [37]:
next_day_predictions[0]

array([   7.4279084 ,    0.11416399,    3.1583736 ,  247.26433   ,
         38.44667   , 1014.9859    ,    4.8776703 ,    0.85280895,
          1.0058229 ], dtype=float32)

In [38]:
# model.save('lstm_model.h5')
model.save('lstm_model.keras')

In [39]:
model.save('lstm_model.h5')

/home/swp/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
